In [1]:
import pandas as pd
import numpy as np
import plotly.express as px 
import plotly.graph_objects as go


In [22]:
d_costdata = pd.read_csv('costs_parameters_nbnlscenariokm.csv')
c10_costdata = pd.read_csv('costs_parameters_10pincrease.csv')
c5_costdata = pd.read_csv('costs_parameters_5pincrease.csv')
c15_costdata = pd.read_csv('costs_parameters_15pincrease.csv')


scenarios = [d_costdata, c5_costdata, c10_costdata, c15_costdata]

In [23]:
#Overview of the Costs for each Group
number= 1

for scenario in scenarios:
    if scenario.equals(d_costdata):
        costs_dict = { 
                    'Group': ["Building and Installations", "Energy Production", "Storage and Conversion", "Carriers", "Infrastructure", "CCUS", "CO2 Emsissions", "Captured Biogenic CO2"],
                    'Default Cost In Billion Euros': [scenario[scenario['Group']=="costs_building_and_installations"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_production"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_storage_and_conversion"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_carriers"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_infrastructure"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_co2"]['Total costs (eur)'].sum(),
                                             scenario['CO2 emission costs (eur)'].sum(),
                                             scenario['Captured biogenic CO2 costs (eur)'].sum()
                                            ]
                    }
        costs_group = pd.DataFrame(costs_dict)
    else:
        costs_dict = { 
                    'Group': ["Building and Installations", "Energy Production", "Storage and Conversion", "Carriers", "Infrastructure", "CCUS", "CO2 Emsissions", "Captured Biogenic CO2"],
                    f'Scenario {number} Cost In Billion Euros': [scenario[scenario['Group']=="costs_building_and_installations"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_production"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_storage_and_conversion"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_carriers"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_infrastructure"]['Total costs (eur)'].sum(),
                                             scenario[scenario['Group']=="costs_co2"]['Total costs (eur)'].sum(),
                                             scenario['CO2 emission costs (eur)'].sum(),
                                             scenario['Captured biogenic CO2 costs (eur)'].sum()
                                            ]
                    }
        costs_group_current = pd.DataFrame(costs_dict)
        costs_group = costs_group.merge(costs_group_current, how='inner', on='Group')
        number += 1

total_values = []

for column in range(len(costs_group.columns)-1):
    total_values.append((costs_group.iloc[:, column+1].sum()/1000000000).round(3))
    costs_group.iloc[:, column+1] = (costs_group.iloc[:, column+1]/1000000000).round(3)

total_values.insert(0, 'Total')
total_df = pd.DataFrame([total_values], columns=costs_group.columns)

costs_group = pd.concat([costs_group, total_df], ignore_index=True)

for column in range(len(costs_group.columns)-2):
    costs_group.insert(3*column+3, f'Change between default and scenario {column+1} (billion euros)', (costs_group.iloc[:, 3*column+2] - costs_group.iloc[:, 1]).round(3))
    costs_group.insert(3*column+4, f'Change between default and scenario {column+1} (in %)', ((costs_group.iloc[:,  3*column+3]/costs_group.iloc[:, 1])*100).round(2))
    costs_group[f'Change between default and scenario {column+1} (in %)'] = (costs_group[f'Change between default and scenario {column+1} (in %)']).astype(str) + '%'

costs_group

,Group,Default Cost In Billion Euros,Scenario 1 Cost In Billion Euros,Change between default and scenario 1 (billion euros),Change between default and scenario 1 (in %),Scenario 2 Cost In Billion Euros,Change between default and scenario 2 (billion euros),Change between default and scenario 2 (in %),Scenario 3 Cost In Billion Euros,Change between default and scenario 3 (billion euros),Change between default and scenario 3 (in %)
0,Building and Installations,13.256,13.189,-0.067,-0.51%,13.121,-0.135,-1.02%,13.189,-0.067,-0.51%
1,Energy Production,13.539,13.568,0.029,0.21%,13.595,0.056,0.41%,13.568,0.029,0.21%
2,Storage and Conversion,18.448,18.447,-0.001,-0.01%,18.446,-0.002,-0.01%,18.447,-0.001,-0.01%
3,Carriers,31.015,31.032,0.017,0.05%,31.053,0.038,0.12%,31.032,0.017,0.05%
4,Infrastructure,20.739,20.745,0.006,0.03%,21.167,0.428,2.06%,20.745,0.006,0.03%
5,CCUS,0.949,0.949,0.000,0.0%,0.949,0.000,0.0%,0.949,0.000,0.0%
6,CO2 Emsissions,0.173,0.173,0.000,0.0%,0.173,0.000,0.0%,0.173,0.000,0.0%
7,Captured Biogenic CO2,-1.963,-1.964,-0.001,0.05%,-1.963,0.000,-0.0%,-1.964,-0.001,0.05%
8,Total,96.157,96.140,-0.017,-0.02%,96.543,0.386,0.4%,96.140,-0.017,-0.02%


In [ ]:
#Buildings and Installations Cost Breakdown into Subgroups
cbi_subgroups = d_costdata[d_costdata['Group'] == "costs_building_and_installations"]
cbi_subgroups = cbi_subgroups.groupby("Subgroup")['Total costs (eur)'].sum()

cbi_total = (cbi_subgroups.sum()/1000000000).round(3)
total_df = pd.DataFrame({'Subgroup': ['Total'], 'Cost In Billion Euros': [cbi_total]})

cbi_subgroupsdata = {'Subgroup': cbi_subgroups.index.tolist(),
                     'Cost In Billion Euros': cbi_subgroups.values.tolist()
                    }
cbi_subgroups = pd.DataFrame(cbi_subgroupsdata)

cbi_subgroups['Cost In Billion Euros'] = (cbi_subgroups['Cost In Billion Euros']/1000000000).round(3)
cbi_subgroups = pd.concat([cbi_subgroups, total_df], ignore_index=True)
cbi_subgroups


In [26]:
#Breakdown of the infrastructure costs
number = 1
for scenario in scenarios:
    inf_scenario = scenario[scenario['Group']=="costs_infrastructure"]

    hydrogen_carriers = inf_scenario[inf_scenario['Subgroup']=="hydrogen"]['Total costs (eur)'].sum() + scenario[scenario['Subgroup']=="liquid_hydrogen"]['Total costs (eur)'].sum() + scenario[scenario['Subgroup']=="lohc"]['Total costs (eur)'].sum()
    natural_gas =       inf_scenario[inf_scenario['Subgroup']=="lng"]['Total costs (eur)'].sum() + scenario[scenario['Subgroup']=="network_gas"]['Total costs (eur)'].sum()
    electricity =       inf_scenario[inf_scenario['Subgroup']=="electricity"]['Total costs (eur)'].sum()
    ammonia =           inf_scenario[inf_scenario['Subgroup']=="ammonia"]['Total costs (eur)'].sum()
    oil_p =             inf_scenario[inf_scenario['Subgroup']=="diesel"]['Total costs (eur)'].sum()
    heat =              inf_scenario[inf_scenario['Subgroup']=="heat"]['Total costs (eur)'].sum()

    if scenario.equals(d_costdata):
        inf_subgroupsdata = {'Subgroup': ["Natural Gas", "Hydrogen Carriers", "Electricity", "Heat", "Ammonia", "Oil and Products"],
                        'Default Cost In Billion Euros': [natural_gas, hydrogen_carriers, electricity, heat, ammonia, oil_p]
                        }
        inf_subgroups = pd.DataFrame(inf_subgroupsdata)
        
    else:
        inf_subgroupsdata = {'Subgroup': ["Natural Gas", "Hydrogen Carriers", "Electricity", "Heat", "Ammonia", "Oil and Products"],
                        f'Scenario {number} Cost In Billion Euros': [natural_gas, hydrogen_carriers, electricity, heat, ammonia, oil_p]
                        }
        inf_subgroups_current = pd.DataFrame(inf_subgroupsdata)
        inf_subgroups = inf_subgroups.merge(inf_subgroups_current, how='inner', on='Subgroup')
        number +=1

total_values = ['Total']

for column in range(len(inf_subgroups.columns)-1):
    total_values.append((inf_subgroups.iloc[:, column+1].sum()/1000000000).round(3))
    inf_subgroups.iloc[:, column+1] = (inf_subgroups.iloc[:, column+1]/1000000000).round(3)

total_df = pd.DataFrame([total_values], columns=inf_subgroups.columns)

inf_subgroups = pd.concat([inf_subgroups, total_df], ignore_index=True)

for column in range(len(inf_subgroups.columns)-2):
    inf_subgroups.insert(3*column+3, f'Change between default and scenario {column+1} (billion euros)', (inf_subgroups.iloc[:, 3*column+2] - inf_subgroups.iloc[:, 1]).round(3))
    inf_subgroups.insert(3*column+4, f'Change between default and scenario {column+1} (in %)', ((inf_subgroups.iloc[:,  3*column+3]/inf_subgroups.iloc[:, 1])*100).round(2))
    inf_subgroups[f'Change between default and scenario {column+1} (in %)'] = (inf_subgroups[f'Change between default and scenario {column+1} (in %)']).astype(str) + '%'

inf_subgroups

,Subgroup,Default Cost In Billion Euros,Scenario 1 Cost In Billion Euros,Change between default and scenario 1 (billion euros),Change between default and scenario 1 (in %),Scenario 2 Cost In Billion Euros,Change between default and scenario 2 (billion euros),Change between default and scenario 2 (in %),Scenario 3 Cost In Billion Euros,Change between default and scenario 3 (billion euros),Change between default and scenario 3 (in %)
0,Natural Gas,1.494,1.494,0.000,0.0%,1.494,0.000,0.0%,1.494,0.000,0.0%
1,Hydrogen Carriers,10.077,10.077,0.000,0.0%,10.077,0.000,0.0%,10.077,0.000,0.0%
2,Electricity,14.867,14.872,0.005,0.03%,15.295,0.428,2.88%,14.872,0.005,0.03%
3,Heat,1.506,1.507,0.001,0.07%,1.506,0.000,0.0%,1.507,0.001,0.07%
4,Ammonia,0.884,0.884,0.000,0.0%,0.884,0.000,0.0%,0.884,0.000,0.0%
5,Oil and Products,0.000,0.000,0.000,nan%,0.000,0.000,nan%,0.000,0.000,nan%
6,Total,28.828,28.833,0.005,0.02%,29.256,0.428,1.48%,28.833,0.005,0.02%


Subcategories hydrogen, liquid_hydrogen, and lohc are combined into one in gquerry under hydrogen_carriers
Subcategories lng and network gas are combined into one category in gquerry under natural_gas
Heat is made up of HT, MT and LT heat
Ammonia is made up of capital and operating expenses
Subcategory diesel appears as oil_and_products in gquery and is breaken down into capital and operating expenses


In [25]:
#Breakdown of the Electricity Infrastructure Costs
number = 1
for scenario in scenarios:
    electricity_scenario = scenario[scenario['Group']=="costs_infrastructure"]
    electricity_scenario = electricity_scenario[electricity_scenario['Subgroup']=="electricity"]

    if scenario.equals(d_costdata):
        electricity_dict = {
            'Electricity Infrastructure Costs' : electricity_scenario['Key'].tolist(),
            'Default Cost In Billion Euros': electricity_scenario['Total costs (eur)'].tolist()
        }
        electricity_infcosts = pd.DataFrame(electricity_dict)
    else:
        electricity_dict =  {
            'Electricity Infrastructure Costs' : electricity_scenario['Key'].tolist(),
            f'Scenario {number} Cost In Billion Euros': electricity_scenario['Total costs (eur)'].tolist()
        }
        electricity_infcosts_current = pd.DataFrame(electricity_dict)
        electricity_infcosts = electricity_infcosts.merge(electricity_infcosts_current, how='inner', on='Electricity Infrastructure Costs')
        number+=1

total_values = []

for column in range(len(electricity_infcosts.columns)-1):
    total_values.append((electricity_infcosts.iloc[:, column+1].sum()/1000000000).round(3))
    electricity_infcosts.iloc[:, column+1] = (electricity_infcosts.iloc[:, column+1]/1000000000).round(3)

total_values.insert(0, 'Total')
total_df = pd.DataFrame([total_values], columns=electricity_infcosts.columns)

electricity_infcosts = pd.concat([electricity_infcosts, total_df], ignore_index=True)

for column in range(len(electricity_infcosts.columns)-2):
    electricity_infcosts.insert(3*column+3, f'Change between default and scenario {column+1} (billion euros)', (electricity_infcosts.iloc[:, 3*column+2] - electricity_infcosts.iloc[:, 1]).round(3))
    electricity_infcosts.insert(3*column+4, f'Change between default and scenario {column+1} (in %)', ((electricity_infcosts.iloc[:,  3*column+3]/electricity_infcosts.iloc[:, 1])*100).round(2))
    electricity_infcosts[f'Change between default and scenario {column+1} (in %)'] = (electricity_infcosts[f'Change between default and scenario {column+1} (in %)']).astype(str) + '%'

electricity_infcosts

,Electricity Infrastructure Costs,Default Cost In Billion Euros,Scenario 1 Cost In Billion Euros,Change between default and scenario 1 (billion euros),Change between default and scenario 1 (in %),Scenario 2 Cost In Billion Euros,Change between default and scenario 2 (billion euros),Change between default and scenario 2 (in %),Scenario 3 Cost In Billion Euros,Change between default and scenario 3 (billion euros),Change between default and scenario 3 (in %)
0,lv_net_costs_present,0.839,0.839,0.000,0.0%,0.839,0.000,0.0%,0.839,0.000,0.0%
1,lv_net_costs_delta_present_future,0.333,0.334,0.001,0.3%,0.335,0.002,0.6%,0.334,0.001,0.3%
2,lv_mv_trafo_costs_present,0.299,0.299,0.000,0.0%,0.299,0.000,0.0%,0.299,0.000,0.0%
3,lv_mv_trafo_costs_delta_present_future,0.113,0.114,0.001,0.88%,0.114,0.001,0.88%,0.114,0.001,0.88%
4,mv_net_costs_present,0.721,0.721,0.000,0.0%,0.721,0.000,0.0%,0.721,0.000,0.0%
5,mv_net_costs_delta_present_future,0.982,0.985,0.003,0.31%,1.039,0.057,5.8%,0.985,0.003,0.31%
6,mv_hv_trafo_costs_present,0.252,0.252,0.000,0.0%,0.252,0.000,0.0%,0.252,0.000,0.0%
7,mv_hv_trafo_costs_delta_present_future,0.430,0.431,0.001,0.23%,0.455,0.025,5.81%,0.431,0.001,0.23%
8,hv_net_costs_present,0.424,0.424,0.000,0.0%,0.424,0.000,0.0%,0.424,0.000,0.0%
9,hv_net_costs_delta_present_future,5.163,5.163,0.000,0.0%,5.506,0.343,6.64%,5.163,0.000,0.0%
